In [59]:
import pandas as pd
import bz2
import json
from urllib.parse import urlparse
import re
import numpy as np
from tld import get_tld

pd.set_option('display.max_colwidth', None) #Print full text
pd.set_option('display.max_rows', 50) #Print full text

In [24]:
FILE_IN = 'quotes-2015.json.bz2'
FILE_OUT = 'processed-quotes-2015.csv.bz2'

In [30]:
json_reader = pd.read_json(FILE_IN,lines=True,chunksize=1_000,compression='bz2') 
LIMIT = 1

for (df_chunk,counter) in zip(json_reader,range(LIMIT)):
    #print(df_chunk.iloc[:10])
    print(df_chunk.columns)
    print(df_chunk['urls'].iloc[:50]) #.apply(lambda url:get_domain(url[0]) ))
    add_header = (counter==0)
    write_mode = 'w' if counter == 0 else 'a'
    df_chunk.to_csv(FILE_OUT,header = add_header,index=False, mode=write_mode,compression='bz2') # writing in the new file

Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [17]:
json_reader = pd.read_csv(FILE_OUT,chunksize=1_000,compression='bz2') 
LIMIT = 10

for (df_chunk,counter) in zip(json_reader,range(LIMIT)):
    print(df_chunk.columns)
    print(df_chunk)

Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
               quoteID                                          quotation  \
0    2015-08-31-000271  ... a great day for veterans here in Littleton...   
1    2015-12-08-029916  How FFA scored 32 own goals in 18 months and C...   
2    2015-09-10-000206  [ Amy ] was placed under an unacceptable amoun...   
3    2015-07-23-032707                    How High Will These Numbers Go?   
4    2015-10-04-000115  [ Ban ] recalls that hospitals and medical per...   
..                 ...                                                ...   
995  2015-08-28-034620  I have never said I would quit this club becau...   
996  2015-11-01-004737  And tomorrow I'm going to be thinking of my mo...   
997  2015-06-24-018085  I have never seen a suspect behave in such a d...   
998  2015-02-02-004250  And unfortunately, with the play that we tried...   
999  2015-07-26-0197

In [ ]:
def process_df(file_in,file_out,process_fn,chunk_to_process=1,chunk_size=1_000):
   
    '''
    
    
    '''
    json_reader = pd.read_json(file_in,lines=True,chunksize=chunk_size,compression='bz2') 
    
    for (df_chunk,counter) in zip(json_reader,range(chunk_to_process)):
        
        new_df_chunk = process_fn(df)
        
        add_header = (counter==0)
        write_mode = 'w' if counter == 0 else 'a'
        new_df_chunk.to_csv(file_out,header = add_header,index=False, mode=write_mode,compression='bz2') # writing to the new file
    

# TEST DATA

In [6]:
FILE_TEST = 'quotes-2019-nytimes.json'
df = pd.read_json(FILE_TEST, lines = True)

## Different number of occurrences and urls

In [149]:
df[df['numOccurrences'] != df['urls'].apply(lambda urls: len(urls))]
df['numUrls'] = df['urls'].apply(lambda urls: len(urls))
print(df[df['numOccurrences'] != df['numUrls']][['numOccurrences','numUrls']].sample(10))
df = df.drop(columns=['numUrls'])
df.columns

        numOccurrences  numUrls
73118               81       71
70060              147      144
75268               12       11
125747              45       24
85022                5        4
69112               21       20
76476               77       54
12132              304      302
102435              61       54
31396              174      161


Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase', 'domains'],
      dtype='object')

## Decide on which authors and  probabilities to keep

In [93]:
top_probas = df['probas'].apply(lambda probas: float(probas[0][1]))
top_probas.describe()
top_probas.values.sort()
(top_probas<=0.5).sum()/len(top_probas) # proportion of top probas below 0.5
#top_probas.quantile(0.1)

0.06715752649052895

## Detect website from url

In [129]:
def extract_domain(url):
    url_pruned = urlparse(url).netloc
    tld = get_tld(url, as_object=True).tld
    url_no_tld = url_pruned.replace('.'+tld,"")
    domain = url_no_tld.split('.')[-1]
    
    return domain

In [130]:
df['domains'] = df['urls'].apply(lambda urls: [extract_domain(url) for url in urls])

In [ ]:
df[['domains','urls','numOccurrences']][:50]

In [93]:
test_url = 'https://nationalpost.com/opinion/robert-fulford-vladimir-nabokovs-remarkable-works-remain-as-powerfu'
extract_domain(test_url)

''

In [128]:
url_pruned = urlparse(test_url).netloc
print(url_pruned)
tld = get_tld(test_url, as_object=True).tld
print(tld)
url_no_tld = url_pruned.replace('.'+tld,"")
print(url_no_tld)
domain = url_no_tld.split('.')[-1]
print(domain)

nationalpost.com
com
nationalpost
nationalpost
